In [2]:
import pandas as pd
import numpy as np
from scipy.sparse import csr_matrix
from scipy.sparse import eye
from scipy.sparse.linalg import inv

from ipynb.fs.full.roc_curve import roc_curve
from sklearn.model_selection import train_test_split

In [3]:
# pd. set_option('display.max_rows', 10) # or 1000.
pd.options.display.max_colwidth = 150

In [4]:
healthy_df = pd.read_excel('features_30_9_healthy.xlsx', header=None)

In [5]:
healthy_df['class'] = 0  # negative
healthy_df.rename(columns={32: 'file_name', 31: 32}, inplace=True)

In [6]:
healthy_df

,0,1,2,3,4,5,6,7,8,9,...,24,25,26,27,28,29,30,32,file_name,class
0,0.09242,0.58207,0.84332,1.85755,0.75574,9.51179,6.68282,3.31333,3.98427,0.32795,...,4539.14893,9787,12757,5068,7689,1721357.125,1312.00500,1,AAL09091956__20181106__12038__CT__HEAD__0002__brain 1 mm_1.2.840.113704.1.111.5520.1541473094.3434_198,0
1,0.06342,0.96902,0.74959,1.93489,0.67394,6.65879,6.44094,3.35085,4.40313,0.47218,...,2038.07898,10136,12932,8039,4893,702499.750,838.15259,1,AAL09091956__20181106__12038__CT__HEAD__0002__brain 1 mm_1.2.840.113704.1.111.5520.1541473094.3434_198,0
2,0.10062,0.68576,0.79684,1.68772,0.73787,9.50464,6.04032,3.17706,3.93885,0.38197,...,4157.31250,9961,12583,5592,6991,1237559.375,1112.45642,1,AAL09091956__20181106__12038__CT__HEAD__0002__brain 1 mm_1.2.840.113704.1.111.5520.1541473094.3434_198,0
3,0.05449,0.74973,0.87516,3.00270,0.71925,7.70397,10.57323,3.66807,4.57908,0.39824,...,3706.43335,8388,12233,4544,7689,2748470.500,1657.85120,1,AAL09091956__20181106__12038__CT__HEAD__0002__brain 1 mm_1.2.840.113704.1.111.5520.1541473094.3434_198,0
4,0.07776,0.70084,0.85559,2.42659,0.73281,8.71716,8.64065,3.42859,4.24003,0.38487,...,4775.73877,9088,12583,3845,8738,2867137.750,1693.26245,1,AAL09091956__20181106__12038__CT__HEAD__0002__brain 1 mm_1.2.840.113704.1.111.5520.1541473094.3434_198,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2103,0.06155,0.69752,0.89054,3.18606,0.74120,7.02580,11.77003,3.62602,4.42574,0.39767,...,3821.18237,5915,10883,2129,8754,3848079.250,1961.65222,1,ZSA29101953__20170511__6862__CT__HEAD__0002__brain 1 mm_1.2.840.113704.1.111.4524.1494468681.17959_237,0
2104,0.06068,0.75961,0.86736,2.86349,0.71831,7.67092,10.40712,3.60249,4.47293,0.40430,...,4307.60449,5678,10646,1656,8990,3530869.500,1879.06079,1,ZSA29101953__20170511__6862__CT__HEAD__0002__brain 1 mm_1.2.840.113704.1.111.4524.1494468681.17959_237,0
2105,0.09644,0.70322,0.85896,2.49303,0.74774,6.01145,9.14703,3.34126,4.10024,0.41432,...,3502.35352,4495,10646,1420,9226,3042845.750,1744.37549,1,ZSA29101953__20170511__6862__CT__HEAD__0002__brain 1 mm_1.2.840.113704.1.111.4524.1494468681.17959_237,0
2106,0.12647,0.66081,0.88106,2.77796,0.75613,5.16639,10.15986,3.13970,3.91218,0.39423,...,2767.90771,4022,10410,1656,8754,3174905.250,1781.82642,1,ZSA29101953__20170511__6862__CT__HEAD__0002__brain 1 mm_1.2.840.113704.1.111.4524.1494468681.17959_237,0


In [7]:
insult_df = pd.read_excel('features_30_9_insult.xlsx', header=None)

In [8]:
insult_df['class'] = 1
insult_df.rename(columns={33: 'file_name'}, inplace=True)

In [9]:
insult_df  # positive

,0,1,2,3,4,5,6,7,8,9,...,25,26,27,28,29,30,31,32,file_name,class
0,0.06081,0.75858,0.85508,2.61725,0.71279,9.07447,9.53269,3.54796,4.38729,0.39242,...,7689,10486,4019,6467,1706812.750,1306.45044,0.09679,0.44111,AAL09091956__20181106__12038__CT__HEAD__0002__brain 1 mm_1.2.840.113704.1.111.5520.1541473094.3434_198,1
1,0.05805,0.80593,0.86047,2.88797,0.70327,8.43171,10.18995,3.56138,4.49190,0.41046,...,7340,10486,4019,6467,1868834.875,1367.05334,0.14702,0.57444,AAL09091956__20181106__12038__CT__HEAD__0002__brain 1 mm_1.2.840.113704.1.111.5520.1541473094.3434_198,1
2,0.05735,0.80487,0.86085,2.89208,0.69992,8.27271,10.48810,3.61427,4.52479,0.40255,...,6990,10311,3845,6466,1798897.125,1341.22974,0.19660,0.60778,AAL09091956__20181106__12038__CT__HEAD__0002__brain 1 mm_1.2.840.113704.1.111.5520.1541473094.3434_198,1
3,0.07138,0.60490,0.87878,2.49509,0.74577,7.20000,9.16320,3.48943,4.20273,0.32976,...,7340,11884,3845,8039,2424746.750,1557.15979,0.24542,0.55333,AAL09091956__20181106__12038__CT__HEAD__0002__brain 1 mm_1.2.840.113704.1.111.5520.1541473094.3434_198,1
4,0.06371,0.61074,0.88761,2.71714,0.74178,8.10054,9.98028,3.60307,4.32808,0.32754,...,8214,12058,3845,8213,2890672.500,1700.19775,0.29329,0.48111,AAL09091956__20181106__12038__CT__HEAD__0002__brain 1 mm_1.2.840.113704.1.111.5520.1541473094.3434_198,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5930,0.06646,0.65228,0.89100,2.99205,0.75269,8.06304,10.62932,3.63314,4.42857,0.38109,...,6861,11120,2129,8991,3640547.750,1908.02197,0.55711,0.62222,ZSA29101953__20170511__6862__CT__HEAD__0002__brain 1 mm_1.2.840.113704.1.111.4524.1494468681.17959_237,1
5931,0.06672,0.58668,0.89720,2.85336,0.75804,8.28487,10.18638,3.62216,4.34481,0.33563,...,6861,11120,1893,9227,3537324.750,1880.77771,0.45430,0.67111,ZSA29101953__20170511__6862__CT__HEAD__0002__brain 1 mm_1.2.840.113704.1.111.4524.1494468681.17959_237,1
5932,0.07679,0.60400,0.87018,2.32624,0.75293,8.35847,8.05221,3.45328,4.18973,0.34140,...,6861,11120,1893,9227,2920900.750,1709.06421,0.34919,0.69000,ZSA29101953__20170511__6862__CT__HEAD__0002__brain 1 mm_1.2.840.113704.1.111.4524.1494468681.17959_237,1
5933,0.07262,0.62511,0.87357,2.47208,0.74652,8.93944,8.51236,3.49029,4.25676,0.34790,...,7098,11120,1893,9227,3107574.750,1762.83142,0.24275,0.63889,ZSA29101953__20170511__6862__CT__HEAD__0002__brain 1 mm_1.2.840.113704.1.111.4524.1494468681.17959_237,1


In [10]:
df = pd.concat([insult_df, healthy_df])

In [11]:
file_names = df['file_name']
unique_file_names = file_names.unique()  # unique file names in df
#df.sort_values(by=['file_name', 'class'], ascending=[True, False], inplace=True)

In [12]:
file_names = df['file_name']  # all file names of healthy_df
unique_file_names = file_names.unique()  # unique file names of healthy_df
df['files_indices'] = file_names.factorize()[0]  # index of unique file names
files_indices_list = df['files_indices']
n_files = len(unique_file_names)  # number of unique files

In [13]:
df

,0,1,2,3,4,5,6,7,8,9,...,26,27,28,29,30,31,32,file_name,class,files_indices
0,0.06081,0.75858,0.85508,2.61725,0.71279,9.07447,9.53269,3.54796,4.38729,0.39242,...,10486,4019,6467,1706812.750,1306.45044,0.09679,0.44111,AAL09091956__20181106__12038__CT__HEAD__0002__brain 1 mm_1.2.840.113704.1.111.5520.1541473094.3434_198,1,0
1,0.05805,0.80593,0.86047,2.88797,0.70327,8.43171,10.18995,3.56138,4.49190,0.41046,...,10486,4019,6467,1868834.875,1367.05334,0.14702,0.57444,AAL09091956__20181106__12038__CT__HEAD__0002__brain 1 mm_1.2.840.113704.1.111.5520.1541473094.3434_198,1,0
2,0.05735,0.80487,0.86085,2.89208,0.69992,8.27271,10.48810,3.61427,4.52479,0.40255,...,10311,3845,6466,1798897.125,1341.22974,0.19660,0.60778,AAL09091956__20181106__12038__CT__HEAD__0002__brain 1 mm_1.2.840.113704.1.111.5520.1541473094.3434_198,1,0
3,0.07138,0.60490,0.87878,2.49509,0.74577,7.20000,9.16320,3.48943,4.20273,0.32976,...,11884,3845,8039,2424746.750,1557.15979,0.24542,0.55333,AAL09091956__20181106__12038__CT__HEAD__0002__brain 1 mm_1.2.840.113704.1.111.5520.1541473094.3434_198,1,0
4,0.06371,0.61074,0.88761,2.71714,0.74178,8.10054,9.98028,3.60307,4.32808,0.32754,...,12058,3845,8213,2890672.500,1700.19775,0.29329,0.48111,AAL09091956__20181106__12038__CT__HEAD__0002__brain 1 mm_1.2.840.113704.1.111.5520.1541473094.3434_198,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2103,0.06155,0.69752,0.89054,3.18606,0.74120,7.02580,11.77003,3.62602,4.42574,0.39767,...,10883,2129,8754,3848079.250,1961.65222,NaN,1.00000,ZSA29101953__20170511__6862__CT__HEAD__0002__brain 1 mm_1.2.840.113704.1.111.4524.1494468681.17959_237,0,45
2104,0.06068,0.75961,0.86736,2.86349,0.71831,7.67092,10.40712,3.60249,4.47293,0.40430,...,10646,1656,8990,3530869.500,1879.06079,NaN,1.00000,ZSA29101953__20170511__6862__CT__HEAD__0002__brain 1 mm_1.2.840.113704.1.111.4524.1494468681.17959_237,0,45
2105,0.09644,0.70322,0.85896,2.49303,0.74774,6.01145,9.14703,3.34126,4.10024,0.41432,...,10646,1420,9226,3042845.750,1744.37549,NaN,1.00000,ZSA29101953__20170511__6862__CT__HEAD__0002__brain 1 mm_1.2.840.113704.1.111.4524.1494468681.17959_237,0,45
2106,0.12647,0.66081,0.88106,2.77796,0.75613,5.16639,10.15986,3.13970,3.91218,0.39423,...,10410,1656,8754,3174905.250,1781.82642,NaN,1.00000,ZSA29101953__20170511__6862__CT__HEAD__0002__brain 1 mm_1.2.840.113704.1.111.4524.1494468681.17959_237,0,45


In [14]:
X0 = df.iloc[:,:31]  # healthy_df feautures

In [15]:
mu = X0.mean()
sig = X0.std()
X = (X0 - mu)/sig
X

,0,1,2,3,4,5,6,7,8,9,...,21,22,23,24,25,26,27,28,29,30
0,-0.658769,-0.388568,0.998650,0.893271,0.168592,1.007406,0.996314,1.102418,0.480440,-0.489884,...,-0.333025,-1.016576,0.676326,-1.419898,-1.606353,-1.792862,-0.994495,-1.588028,-1.001290,-1.372388
1,-0.768904,-0.176755,1.068378,1.325875,-0.045791,0.465373,1.266773,1.169118,0.840055,-0.277057,...,-0.589243,-1.096587,0.822323,-1.452464,-1.638076,-1.792862,-0.994495,-1.588028,-0.995633,-1.349013
2,-0.796837,-0.181497,1.073294,1.332442,-0.121230,0.331290,1.389460,1.431993,0.953120,-0.370375,...,-0.616231,-1.059719,0.721063,-1.467731,-1.669890,-1.803664,-1.017111,-1.588102,-0.998075,-1.358973
3,-0.236982,-1.076033,1.305248,0.698062,0.911277,-0.573315,0.844271,0.811511,-0.154018,-1.229118,...,-0.706493,-1.333484,1.291670,-1.424115,-1.638076,-1.706570,-1.017111,-1.471346,-0.976223,-1.275687
4,-0.543047,-1.049908,1.419478,1.052893,0.821425,0.186101,1.180495,1.376327,0.276895,-1.255308,...,-0.629795,-1.619799,1.687822,-1.348647,-1.558632,-1.695829,-1.017111,-1.458430,-0.959956,-1.220516
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2103,-0.629240,-0.661711,1.457382,1.802215,0.808364,-0.720216,1.916966,1.490393,0.612618,-0.427947,...,-0.879920,-1.132671,0.862539,-1.394970,-1.767604,-1.768357,-1.240160,-1.418275,-0.926528,-1.119671
2104,-0.663956,-0.383961,1.157511,1.286756,0.292898,-0.176193,1.356137,1.373444,0.774842,-0.349730,...,-0.723328,-0.971079,0.746437,-1.326298,-1.789147,-1.782986,-1.301641,-1.400757,-0.937603,-1.151527
2105,0.763015,-0.636213,1.048844,0.694770,0.955640,-1.575607,0.837618,0.075074,-0.506346,-0.231518,...,-0.857973,-0.356089,0.337741,-1.439981,-1.896678,-1.782986,-1.332317,-1.383240,-0.954642,-1.203477
2106,1.961335,-0.825928,1.334743,1.150081,1.144576,-2.288238,1.254391,-0.926722,-1.152835,-0.468531,...,-0.981808,0.668371,-0.275668,-1.543667,-1.939672,-1.797553,-1.301641,-1.418275,-0.950032,-1.189031


In [16]:
# feature_32 = df.iloc[:,31]  # вероятности больных по расст от центра
# feature_33 = df.iloc[:,32]  # вероятности больных по пересеч с маской

In [17]:
# feature_33.where(feature_33<0.95, other=1, inplace=True)
# big_prob_feature_33 = feature_33 > 0.05  # there is no feature less than 0.05
# feature_33 = feature_33[big_prob_feature_33]
# y = feature_33

In [53]:
y = df['class']

In [54]:
X_train = X[df['files_indices']<23]
X_test = X[df['files_indices']>23]
y_train = y[df['files_indices']<23]
y_test = y[df['files_indices']>23]
#X_train, X_test, y_train, y_test = train_test_split(X, y.values, test_size=0.50, random_state=1, stratify=y)

In [55]:
X_train_lab = X[(df['class']==1) & (df['files_indices']<23)]
X_test_unlab = X[(df['class']==0) & (df['files_indices']>23)]
y_train_lab = y[(df['class']==1) & (df['files_indices']<23)]
y_test_unlab = y[(df['class']==0) & (df['files_indices']>23)]

In [74]:
X_train_mixed = np.concatenate((X_train_lab, X_test_unlab))

In [98]:
def roc_curve(class_1, class_2, disp_data=1):  # negative, positive
#     rng(1337,'twister')
#     s = rng
#     class_1 = rand(1,3);
#     rng(s)
#     class_2 = 0.5*rand(1,5);
#     sispt = 1;
#     dispp = 1;

#     np.random.seed(1337)
#     class_1 = np.random.rand(1,3)
#     np.random.seed(1337)
#     class_2 = 0.5*np.random.rand(1,3)

    # Calculating the threshold values between the data points
    s_data = np.sort(np.unique(np.concatenate([class_1, class_2], axis=0)))
    s_data = s_data[s_data!=np.array(None)]
    d_data = np.diff(s_data)
    if np.all(d_data==0):
        print('Both class are the same!')
    d_data = np.append(d_data, d_data[-1])
    thres = np.empty(len(s_data)+1)
    thres[0] = s_data[0] - d_data[0]
    thres[1:len(thres)] = s_data + d_data/2

    # Calculating the sensibility and specificity of each threshold
    curve = np.zeros((len(thres),2))
    distance = np.zeros((len(thres),1))
    for id_t in range(len(thres)):
        TP = np.sum(class_2 >= thres[id_t])
        FP = np.sum(class_1 >= thres[id_t])
        FN = np.sum(class_2 < thres[id_t])
        TN = np.sum(class_1 < thres[id_t])

        curve[id_t, 0] = TP/(TP+FN)  # sensitivity - TPR
        curve[id_t, 1] = TN/(TN+FP)  # specificity - FPR

        distance[id_t] = np.sqrt((1-curve[id_t, 0])**2 + (curve[id_t,1]-1)**2)

    # Optimum threshold and parameters
    #opt = np.min(distance)  # 1
    opt = np.argmin(distance)
    TP = np.sum(class_2 >= thres[opt])
    FP = np.sum(class_1 >= thres[opt])
    FN = np.sum(class_2 < thres[opt])
    TN = np.sum(class_1 < thres[opt])

    if disp_data==1:
        print('\n ROC CURVE PARAMETERS:\n')
        print('Threshold:', thres[opt])
        print('Distance:', distance[opt])
        print('Sensitivity:', curve[opt,0])
        print('Specificity:', curve[opt,1])
        print('Accuracy:', (TP+TN)/(TP+TN+FP+FN))
        print('PPV:', TP/(TP+FP))
        print('NPV:', TN/(TN+FN))
        print('FNR:', FN/(FN+TP))
        print('FPR:', FP/(FP+TN))
        print('FDR:', FP/(FP+TP))
        print('FOR:', FN/(FN+TN))
        print('F1 score:', 2*TP/(2*TP+FP+FN))
        print('TP:', TP)
        print('FP:', FP)
        print('FN:', FN)
        print('TN:', TN)

# Decision Tree

In [82]:
from sklearn.semi_supervised import LabelPropagation
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

nolabel = [-1 for _ in range(len(y_test_unlab))]
y_train_mixed = np.concatenate((y_train_lab, nolabel))

# define model
model = LabelPropagation()
# fit model on training dataset
model.fit(X_train_mixed, y_train_mixed)
# get labels for entire training dataset data
tran_labels = model.transduction_

clf = DecisionTreeClassifier()

# Train Decision Tree Classifer
clf = clf.fit(X_train_mixed, tran_labels)

#Predict the response for test dataset
y_pred = clf.predict(X_test)

# calculate score for test set
print("Accuracy:",accuracy_score(y_test, y_pred))

C:\Users\Kunasyl.DESKTOP-N9JUTUQ\anaconda3\lib\site-packages\sklearn\semi_supervised\_label_propagation.py:281: RuntimeWarning: invalid value encountered in true_divide
  self.label_distributions_ /= normalizer


Accuracy: 0.7299864314789688


C:\Users\Kunasyl.DESKTOP-N9JUTUQ\anaconda3\lib\site-packages\sklearn\semi_supervised\_label_propagation.py:290: ConvergenceWarning: max_iter=1000 was reached without convergence.
  warnings.warn(


In [85]:
from sklearn.semi_supervised import LabelPropagation
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score

nolabel = [-1 for _ in range(len(y_test_unlab))]
y_train_mixed = np.concatenate((y_train_lab, nolabel))

# define model
model = LabelPropagation()
# fit model on training dataset
model.fit(X_train_mixed, y_train_mixed)
# get labels for entire training dataset data
tran_labels = model.transduction_

clf = GaussianNB()

# fit the model with the training data
clf.fit(X_train_mixed, tran_labels)

# predict the target on the test dataset
predict_test = clf.predict(X_test)
print('Target on test data',predict_test) 

# Accuracy Score on test dataset
accuracy_test = accuracy_score(y_test,predict_test)
print('accuracy_score on test dataset : ', accuracy_test)

C:\Users\Kunasyl.DESKTOP-N9JUTUQ\anaconda3\lib\site-packages\sklearn\semi_supervised\_label_propagation.py:281: RuntimeWarning: invalid value encountered in true_divide
  self.label_distributions_ /= normalizer


Target on test data [1 1 1 ... 1 1 1]
accuracy_score on test dataset :  0.7299864314789688


C:\Users\Kunasyl.DESKTOP-N9JUTUQ\anaconda3\lib\site-packages\sklearn\semi_supervised\_label_propagation.py:290: ConvergenceWarning: max_iter=1000 was reached without convergence.
  warnings.warn(


# SVM

In [90]:
from sklearn.semi_supervised import LabelSpreading
from sklearn import svm

y_train_mixed = np.concatenate((y_train_lab, y_test_unlab))
#y_train_mixed = np.concatenate((y_train_lab, nolabel))

model = LabelSpreading()
# fit model on training dataset
model.fit(X_train_mixed, y_train_mixed)
# get labels for entire training dataset data
tran_labels = model.transduction_

support = svm.SVC(kernel='rbf')

# Train the model using the training sets and check score on test dataset
support.fit(X_train_mixed, tran_labels)
predicted= support.predict(X_test)
score=accuracy_score(y_test,predicted)
print("Your Model Accuracy is", score)

Your Model Accuracy is 0.7731343283582089
